In [4]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 

from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol
import matplotlib.pyplot as plt 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def get_positional_dbs(circuit, circuit_db):

    qubits_involved = get_qubits_involved(circuit, circuit_db)
    
    gates_on_qubit = {q:[] for q in qubits_involved}
    on_qubit_order = {q:[] for q in qubits_involved}

    for order_gate, ind_gate in enumerate( circuit_db["ind"]):
        if ind_gate < translator.number_of_cnots:
            control, target = translator.indexed_cnots[str(ind_gate)]
            gates_on_qubit[control].append(ind_gate)
            gates_on_qubit[target].append(ind_gate)
            on_qubit_order[control].append(order_gate)                
            on_qubit_order[target].append(order_gate)  
        else:
            gates_on_qubit[(ind_gate-translator.n_qubits)%translator.n_qubits].append(ind_gate)
            on_qubit_order[(ind_gate-translator.n_qubits)%translator.n_qubits].append(order_gate)        
    return gates_on_qubit, on_qubit_order

In [9]:
def rule_1(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path):
            if ind_gate < translator.number_of_cnots:
                control, target = translator.indexed_cnots[str(ind_gate)]
                if (q == control) and (order_gate_on_qubit == 0):
                    pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                    
                    block_id = circuit_db.iloc[pos_gate_to_drop]["block_id"]
                    simplified_db.loc[int(pos_gate_to_drop)+0.1] = gate_template(translator.number_of_cnots + translator.n_qubits + control, param_value=0.0, block_id=circuit_db.iloc[0]["block_id"])
                    simplified_db.loc[int(pos_gate_to_drop)+0.11] = gate_template(translator.number_of_cnots + translator.n_qubits + target, param_value=0.0, block_id=circuit_db.iloc[0]["block_id"])
                    
                    simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                    
                    simplification = True
                    break
    simplified_db = simplified_db.sort_index().reset_index(drop=True)
    return simplification, simplified_db


def rule_2(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            
            next_ind_gate = qubit_gates_path[order_gate_on_qubit+1]
            if (ind_gate < translator.number_of_cnots) and (ind_gate == next_ind_gate):
                control, target = translator.indexed_cnots[str(ind_gate)]
                not_gates_in_between = False
                this_qubit = q
                other_qubits = [control, target]
                other_qubits.remove(q)
                other_qubit = other_qubits[0]
                
                ## now we need to check what happens in the other_qubit
                for qord_other, ind_gate_other in enumerate(gates_on_qubit[other_qubit][:-1]):
                    if (ind_gate_other == ind_gate) and (gates_on_qubit[other_qubit][qord_other +1] == ind_gate):
                        ## if we append the CNOT for q and other_q on the same call, and also for the consecutive
                        ## note that in between there can be other calls for other qubits
                        order_call_q = on_qubit_order[q][order_gate_on_qubit]
                        order_call_other_q = on_qubit_order[other_qubit][qord_other]
                        
                        order_call_qP1 = on_qubit_order[q][order_gate_on_qubit+1]
                        order_call_other_qP1 = on_qubit_order[other_qubit][qord_other+1]
                        
                        ## then it's kosher to say they are consecutively applied (if only looking at the two qubits)
                        if (order_call_q == order_call_other_q) and (order_call_qP1 == order_call_other_qP1):
                            
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit+1]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)

                            simplification = True
                            break
                if simplification is True:
                    break
            else:       
                simplification = False
                gate_analyzed[q][order_gate_on_qubit] = True
    simplified_db = simplified_db.reset_index(drop=True)
    return simplification, simplified_db



def apply_rule(original_circuit_db, rule, **kwargs):
    max_cnt = kwargs.get('max_cnt',10)
    simplified, cnt = True, 0
    original_circuit, original_circuit_db = translator.give_circuit(original_circuit_db)
    gates_on_qubit, on_qubit_order = get_positional_dbs(original_circuit, original_circuit_db)
    simplified_db = original_circuit_db.copy()
    while simplified and cnt < max_cnt:
        simplified, simplified_circuit_db = rule(translator, simplified_db, on_qubit_order, gates_on_qubit)
        circuit, simplified_db = translator.give_circuit(simplified_circuit_db)
        gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, simplified_db)
        cnt+=1
    return cnt, simplified_db

In [10]:
translator = CirqTranslater(3)
db0 = pd.DataFrame([gate_template(translator.cnots_index[str([q,q+1])], block_id=0) for q in range(translator.n_qubits -1)])
db1 = pd.DataFrame([gate_template(translator.cnots_index[str([q,q+1])], block_id=0) for q in range(translator.n_qubits -1) for j in range(2)])
circuit_db = concatenate_dbs([db0, db1])
circuit, circuit_db = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)

In [13]:
circuit_db["symbol"].

0    None
1    None
2    None
3    None
4    None
5    None
Name: symbol, dtype: object

In [198]:
cnts, simplified_db = apply_rule(circuit_db, rule_1)

In [201]:
circuit, circuit_db = translator.give_circuit(simplified_db)

In [202]:
cnts, simplified_db = apply_rule(circuit_db, rule_2)

In [203]:
circuit, circuit_db = translator.give_circuit(simplified_db)

In [204]:
circuit

(0, 0): ───Rx(0)───────

(0, 1): ───Rx(0)───@───
                   │
(0, 2): ───────────X───